# Phân loại tình cảm của các câu từ tập dữ liệu Rotten Tomatoes

## Import modules

In [9]:
import numpy as np
import pandas as pd

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

# package used to creat wordclouds
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [11]:
# Scikit-learn is a free software machine learning library for the Python programming language.

# Some classes help us to deal with data
from sklearn.feature_extraction.text import TfidfVectorizer

# Some classes help us to implement ML methods and evaluate models
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

In [1]:
# Keras is a high-level neural networks API

# Some classes help us to deal with data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

ModuleNotFoundError: No module named 'tensorflow'